****This is the SVM Testing python notebook, everything needed to test results is contained within this notebook****


In [ ]:
import cv2
from labeling.image_load_util import load_folder_image, get_full_path_from_root
from utils.serialize import read_clusters, read_matrix
import os

# This cell takes the slic results and loads them for use
matrix_file = open("data/slic/full_slic_matrix.bin", "rb")
cluster_file = open("data/slic/full_slic_cluster.bin", "rb")
print("Files loaded")

# Load the matrix and cluster
matrix = read_matrix(matrix_file) # Matrix size image, pixel corresponds to cluster
cluster = read_clusters(cluster_file) # Cluster ID, avg colour and centroid. 

image_directory = 'src/classification/data/raw/test-images/leftImg8bit/train'

image_filenames =  load_folder_image(image_directory)

id_directory = 'src/classification/data/raw/test-images/gtFine/train' # This is the ground truth labels
search_string = 'label'  # Replace this with the substring you want to match
id_filenames = []

all_filenames = load_folder_image(id_directory)
for id in all_filenames:
    if search_string in str(id):
        id_filenames.append(id)


matrix_file.close()
cluster_file.close()

In [ ]:
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import dump, load
from joblib import Parallel, delayed

def extract_statistical_features(image):
    # Split channels and compute mean, std, median, min, and max using list comprehension.
    return np.array([
        [np.mean(ch), np.std(ch), np.median(ch), np.min(ch), np.max(ch)]
        for ch in cv2.split(image)
    ]).flatten()

def extract_features_from_clusters(cluster_matrix):
    # Combine mean color and centroid for each cluster using list comprehension.
    return np.array([
        [cluster.l, cluster.a, cluster.b, cluster.x, cluster.y]
        for cluster in cluster_matrix
    ])

def process_cluster(cluster, gt, image, slic):
    # Computes all relevant features for cluster and the dominant ground truth label, fully modular.
    cid = cluster.cid
    mask = (slic == (cid + 1))
    mask_uint8 = mask.astype(np.uint8) * 255
    masked_image = cv2.bitwise_and(image, image, mask=mask_uint8)
    stats = extract_statistical_features(masked_image)
    feature_vector = np.concatenate((
        [cluster.l, cluster.a, cluster.b, cluster.x, cluster.y],
        stats
    ))
    labels, counts = np.unique(gt[mask], return_counts=True)
    if counts.size == 0:
        # Set a default label or skip processing as needed
        dominant_label = -1
    else:
        dominant_label = labels[np.argmax(counts)]
    return feature_vector, dominant_label

def prepare_training_data(cluster_matrices, ground_truth_matrices, slic_matrix, images):
    results = []
    # Iterate over each image's data
    for j, (clusters, gt, image, slic) in enumerate(zip(cluster_matrices, ground_truth_matrices, images, slic_matrix)):
        # Parallel processing for all clusters in the current image
        clusters_result = Parallel(n_jobs=-1)(
            delayed(process_cluster)(cluster, gt, image, slic) for cluster in clusters
        )
        results.extend(clusters_result)
    X_train, y_train = zip(*results)
    return np.array(X_train), np.array(y_train)

def train_svm(X_train, y_train):
    # Create a pipeline with StandardScaler and SVC.
    pipeline = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=10, gamma='scale', probability=True))
    pipeline.fit(X_train, y_train)
    return pipeline

def process_cluster_pre(cluster, segments, image):
    # Computes all relevant features for cluster, fully modular.
    cid = cluster.cid
    mask = (segments == (cid + 1))
    mask_uint8 = mask.astype(np.uint8) * 255
    masked_image = cv2.bitwise_and(image, image, mask=mask_uint8)
    stats = extract_statistical_features(masked_image)
    feature_vector = np.concatenate((
        np.array([cluster.l, cluster.a, cluster.b, cluster.x, cluster.y]),
        stats
    ))
    return cluster, feature_vector

def predict_cluster_labels(cluster_matrix, model, segments, image):
    results = Parallel(n_jobs=-1)(
        delayed(process_cluster_pre)(cluster, segments, image) for cluster in cluster_matrix
    )
    clusters, features = zip(*results)
    predictions = model.predict(features)
    segmentation_result = np.zeros(segments.shape, dtype=np.int32)
    for cluster, label in zip(clusters, predictions):
        segmentation_result[segments == (cluster.cid + 1)] = label
    return segmentation_result

In [ ]:
cluster_cell = []
matrix_cell = []
matrix_file = open("data/slic/full_slic_matrix.bin", "rb")
cluster_file = open("data/slic/full_slic_cluster.bin", "rb")


for i in range(0,100):
    cluster_cell.append(read_clusters(cluster_file))
    matrix_cell.append(read_matrix(matrix_file))



print(matrix_cell[0])
print(cluster_cell[0][10])
matrix_file.close()
cluster_file.close()

In [ ]:
id_directory = 'src/classification/data/raw/test-images/gtFine/train'
id_directory_photos = 'src/classification/data/raw/test-images/leftImg8bit/train'
# search_string = 'instance'  # Replace this with the substring you want to match
search_string = 'label'  # Replace this with the substring you want to match
id_filenames = []
id_filenames_photos = []

all_filenames = load_folder_image(id_directory)
for id in all_filenames:
    if search_string in str(id):
        id_filenames.append(id)

all_filenames = load_folder_image(id_directory_photos)
for id in all_filenames:
    id_filenames_photos.append(cv2.imread(id, cv2.IMREAD_COLOR))

ground_truth_matrices = []
for id_filename in id_filenames:
    ground_truth_matrices.append(cv2.imread(id_filename, cv2.IMREAD_GRAYSCALE))

In [ ]:
print(np.shape(ground_truth_matrices))
print(np.shape(id_filenames_photos))

In [ ]:
matrix_file = open("data/slic/full_slic_matrix.bin", "rb")
cluster_file = open("data/slic/full_slic_cluster.bin", "rb")
appended_gt = []
matrix_cell = []
cluster_cell = []
image_cell = []


for i in range(0,50):
    cluster_cell.append(read_clusters(cluster_file))
    matrix_cell.append(read_matrix(matrix_file))
    appended_gt.append(ground_truth_matrices[i])
    image_cell.append(id_filenames_photos[i])


x_train, y_train = prepare_training_data(cluster_cell, appended_gt, matrix_cell,image_cell)
matrix_file.close()
cluster_file.close()

In [ ]:
# print(x_train)
print(np.shape(x_train))

In [ ]:
model = train_svm(x_train[0:2500], y_train[0:2500]) # Train on the first 2500 samples

In [ ]:
test_image = 30 # 9 at 77.5%, 11, 15,16, 3, 18, 22
predicted_labels = predict_cluster_labels(cluster_cell[test_image], model, matrix_cell[test_image],id_filenames_photos[test_image])

In [ ]:
def calculate_matching_percentage(matrix1, matrix2):
    # Ensure both matrices have the same shape
    if np.shape(matrix1) != np.shape(matrix2):
        raise ValueError("Matrices must have the same shape")

    # Compare the matrices element-wise
    matching_elements = (matrix1 == matrix2)

    # Calculate the percentage of matching values
    matching_percentage = np.sum(matching_elements) / matching_elements.size * 100

    return matching_percentage

In [ ]:
def label_consolidate(matrix):
    matrix = np.where((7 <= matrix) & (matrix <= 10), 7, matrix)
    matrix = np.where((11 <= matrix) & (matrix <= 16), 11, matrix)
    matrix = np.where((17 <= matrix) & (matrix <= 20), 17, matrix)
    matrix = np.where((21 <= matrix) & (matrix <= 22), 21, matrix)
    matrix = np.where((24 <= matrix) & (matrix <= 25), 24, matrix)
    matrix = np.where((matrix < 0) | (matrix > 25), 33, matrix)
    return matrix

def label_consolidate_colour(matrix):
    rgb_image = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Define masks
    masks = [
        ((0 <= matrix) & (matrix <= 6),       [111, 74, 0]),
        ((7 <= matrix) & (matrix <= 10),      [52, 152, 219 ]),
        ((11 <= matrix) & (matrix <= 16),     [22, 160, 133 ]),
        ((17 <= matrix) & (matrix <= 20),     [243, 156, 18 ]),
        ((21 <= matrix) & (matrix <= 22),     [249, 231, 159 ]),
        ((24 <= matrix) & (matrix <= 25),     [165, 105, 189 ]),
        ((matrix < 0) | (matrix > 25),        [169, 50, 38])
    ]

    # Assign colors
    for mask, color in masks:
        rgb_image[mask] = color

    return rgb_image

In [ ]:
# Less Restricted labels
predicted_image = label_consolidate(predicted_labels)
gt_image = label_consolidate(appended_gt[test_image])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.patches import Patch

bounding_image = label_consolidate_colour(predicted_labels)
image = id_filenames_photos[test_image]

plt.figure(figsize=(8, 8))
plt.imshow(image)  # Display original image
plt.show()
plt.figure(figsize=(8, 8))
# Define custom legend
legend_elements = [
    Patch(facecolor=np.array([111, 74, 0])/255,      label='Unlabelled'),
    Patch(facecolor=np.array([52, 152, 219])/255,    label='Road/Ground'),
    Patch(facecolor=np.array([22, 160, 133])/255,    label='Building/Wall'),
    Patch(facecolor=np.array([243, 156, 18])/255,    label='Pole/Traffic Light'),
    Patch(facecolor=np.array([249, 231, 159])/255,   label='Vegetation'),
    Patch(facecolor=np.array([165, 105, 189])/255,   label='Person'),
    Patch(facecolor=np.array([169, 50, 38])/255,     label='Vehicle')
]

# Display the legend
plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.axis('off')  # Hide axis for cleaner visualization
plt.tight_layout()
plt.imshow(bounding_image, alpha=0.5)  # Overlay bounding image with 50% transparency
plt.axis('off')  # Optional: Hide axis ticks
plt.show()

In [ ]:
percentage = calculate_matching_percentage(predicted_image, gt_image)

print(f"Matching percentage: {percentage:.2f}%")

In [ ]:
from utils.segmentation_utils import segmented_to_color

# Display Segmented image with color
color_segmented_matrix = segmented_to_color(matrix_cell[test_image], cluster_cell[test_image])
plt.imshow(color_segmented_matrix)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(predicted_image, cmap='gray')
plt.colorbar()
plt.imshow(gt_image, cmap='gray')
plt.colorbar()

In [ ]:
avg_accuracy = 0
max_iter = 50

# Define custom legend
legend_elements = [
    Patch(facecolor=np.array([111, 74, 0])/255,      label='Unlabelled'),
    Patch(facecolor=np.array([52, 152, 219])/255,    label='Road/Ground'),
    Patch(facecolor=np.array([22, 160, 133])/255,    label='Building/Wall'),
    Patch(facecolor=np.array([243, 156, 18])/255,    label='Pole/Traffic Light'),
    Patch(facecolor=np.array([249, 231, 159])/255,   label='Vegetation'),
    Patch(facecolor=np.array([165, 105, 189])/255,   label='Person'),
    Patch(facecolor=np.array([169, 50, 38])/255,     label='Vehicle')
]

for i in range(0,max_iter):
    predicted_labels = predict_cluster_labels(cluster_cell[i], model, matrix_cell[i], id_filenames_photos[i])
    predicted_image = label_consolidate(predicted_labels)
    bounding_image = label_consolidate_colour(predicted_labels)
    gt_image_test = label_consolidate(appended_gt[i])
    percentage = calculate_matching_percentage(predicted_image, gt_image_test)
    avg_accuracy += percentage
    print(f"At iteration {i} matching percentage: {percentage:.2f}%")
    plt.figure(figsize=(8, 8))
    plt.imshow(id_filenames_photos[i])
    plt.show()
    plt.close()
    plt.figure(figsize=(8, 8))
    plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    plt.axis('off')  # Hide axis for cleaner visualization
    plt.tight_layout()
    plt.imshow(bounding_image)
    plt.show()
    plt.close()
    
avg_accuracy = avg_accuracy/max_iter
print(f"Average accuracy: {avg_accuracy:.2f}%")

In [ ]:
print(avg_accuracy)